In [1]:
from pyspark.sql import SparkSession
import numpy as np

In [2]:
spark = SparkSession.builder.appName("CSCI316-process_two_random_forest(simple)") \
.config("spark-master", "local") \
.getOrCreate()

df_FD = spark \
.read \
.format("csv") \
.option("header", "true").load("block_1.csv")

df_FD.printSchema()
df_FD.show(10)

root
 |-- id_1: string (nullable = true)
 |-- id_2: string (nullable = true)
 |-- cmp_fname_c1: string (nullable = true)
 |-- cmp_fname_c2: string (nullable = true)
 |-- cmp_lname_c1: string (nullable = true)
 |-- cmp_lname_c2: string (nullable = true)
 |-- cmp_sex: string (nullable = true)
 |-- cmp_bd: string (nullable = true)
 |-- cmp_bm: string (nullable = true)
 |-- cmp_by: string (nullable = true)
 |-- cmp_plz: string (nullable = true)
 |-- is_match: string (nullable = true)

+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|37291|53113|0.833333333333333|           ?|           1|           ?|      1|     1|     1|     1|      0|    TRUE|
|39086|47614|                1|           ?|  

In [3]:
##following is to do preprocessing
from pyspark.sql.functions import when   
from pyspark.sql.functions import regexp_replace,col
df_FD = df_FD.withColumn('is_match', regexp_replace(col('is_match'), "FALSE", "1"))
df_FD = df_FD.withColumn('is_match', regexp_replace(col('is_match'), "TRUE", "0"))

In [4]:
#convert each tuples of RDD to list
rdd1 = df_FD.rdd.map(list)
rdd1.first()

['37291',
 '53113',
 '0.833333333333333',
 '?',
 '1',
 '?',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0']

In [5]:
#delete first two columns and covert "?" to integer 2 ,otherwise float it. 
def preprocessing(pieces):
    
    scores = [ 2 if p=='?' else float(p) for p in pieces[2:12]]
    
    return scores

#convert rdd to list type
dataset = rdd1.map(lambda x: preprocessing(x)).collect()


record_linkage = dataset

print(type(record_linkage))
print(type(dataset))
print(record_linkage[:1])

<class 'list'>
<class 'list'>
[[0.833333333333333, 2, 1.0, 2, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]]


In [17]:
import copy
from random import seed
from random import randrange
from math import sqrt
from csv import reader

# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset


#simple random sampling with replacement
def subsample(dataset, ratio):

    sample = []
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample


#split a dataset based on an attribute and an attribute value
def test_split(dataset, index, value):

    left  = []
    right = []
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right


#calculate the Gini index for a split dataset
def gini_index(groups, class_values):

    gini = 0.0
    for class_value in class_values:
        for group in groups:
            size = len(group)
            if size == 0:
                continue
            proportion = [row[-1] for row in group].count(class_value) / float(size)
            gini += (proportion * (1.0 - proportion))
    return gini

def IG_index(groups, class_values):
    
    shannonEnt = 0.0
    for class_value in class_values:
        for group in groups:
            size = len(group)
            if size == 0:
                continue
                
            proportion = [row[-1] for row in group].count(class_value) / float(size)
            
            if (proportion == 0 or (1.0 - proportion) == 0):
                return 0
            shannonEnt = -(1.0 - proportion) * np.log2(1.0 - proportion) - proportion * np.log2(proportion)
                       
    return shannonEnt

#create child splits for a node or make terminal
def get_split(dataset, n_features):

    # randomly selected n features
    features = []
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features.append(index)

    # get set of uniq labels
    class_values = list(set(row[-1] for row in dataset))

    # init params
    b_score  = 9999  # minimum Gini index
    b_index  = None  # index of best column
    b_value  = None  # best cut-off value of the best column
    b_groups = None  # best groups

    # loop through selected features to get the minimum Gini index
    for col_index in features:
        for row in dataset:
            col_value = row[col_index]
            groups = test_split(dataset, col_index, col_value)
            gini = IG_index(groups, class_values)
            if gini < b_score:
                b_index  = col_index
                b_value  = col_value
                b_score  = gini
                b_groups = groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}


#create child splits for a node or make terminal
def to_terminal(group):

    labels = [row[-1] for row in group]
    return max(set(labels), key=labels.count)


#create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):

    left, right = node['groups']
    del(node['groups'])

    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return

    # check for max depth
    if depth >= max_depth:
        node['left']  = to_terminal(left)
        node['right'] = to_terminal(right)
        return

    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)

    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)


def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root


#split the dataset into n folds
def predict(node, row):

    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']


#split the dataset into n folds
def bagging_predict(trees, row):

    predictions = [predict(tree, row) for tree in trees]
    # return the label that is voted by most of the trees
    return max(set(predictions), key=predictions.count)


def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    # train
    trees = []
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
    # predict
    
    predictions = [bagging_predict(trees, row) for row in test]
    print(trees)
    print(type(predictions))
    return(predictions)


#split the dataset into n folds
def cross_validation_split(dataset, n_folds):

    dataset_copy = copy.copy(dataset)
    fold_size = int(len(dataset_copy)/n_folds)
    folds = []
    for i in range(n_folds):
        fold = []
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        folds.append(fold)
    return folds

 
#calculate accuracy percentage
def accuracy_metric(actual, predicted):

    correct = 0
    
    label = actual

    pred = predicted
    conf_mat = np.zeros([2, 2])
    for i in range(len(actual)):
         row = int(1 - label[i])
         col = int(1 - pred[i])
         conf_mat[row][col] += 1        
        
         if actual[i] == predicted[i]:
            correct += 1
            
    TP = conf_mat[0][0]
    FP = conf_mat[1][0]
    FN = conf_mat[0][1]
    TN = conf_mat[1][1]
    P = conf_mat[0].sum()
    N = conf_mat[1].sum()
    All = P + N
    Precision=TP / (TP + FP)
    Recall=TP /(TP + FN)
    
    print(" ")
    print("Confusion matrix:")
    print("\t", conf_mat[0])
    print("\t", conf_mat[1])
    print("\tAcc: ", (TP + TN) / All)
    print("\tPrecision : ", TP / (TP + FP))
    print("\tRecall: ",TP /(TP + FN))
    print("\tF1-score: ",2*(Recall * Precision) / (Recall + Precision))
    print("-------------------------")
    print()
    print("\tAccuracy from scratch: ",(TP + TN) / All)
        
    return correct / float(len(actual)) * 100.0


def evaluate_algorithm(dataset, algorithm, params, n_folds=2):
    folds = cross_validation_split(dataset, n_folds)
    scores = []
    for fold in folds:
        # prepare train set
        trainset = copy.copy(folds)
        trainset.remove(fold)
        trainset = sum(trainset, []) # flatten trainset
        # prepare test set
        testset = copy.copy(fold)
        # train & predict
        predicted = algorithm(trainset, testset, **params)
        # evaluate
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
        
             
        
    return scores


if __name__ == '__main__':
    seed(1)
    
    dataSet = record_linkage
    #because original dataset spent pretty pretty much time,so I have to reduce data size by randon selecting--- 
    
    ratio=0.01 #can set ratio of remaining data
    
    reduced_dataSet_number=int(ratio*len(dataSet))
    
    print("numer of samples: " +  str(reduced_dataSet_number))
    
    shuffled_indices=np.random.permutation(reduced_dataSet_number)
    
    reduced_index=shuffled_indices[:reduced_dataSet_number]
    
    print(reduced_index)
    reduced_data=[dataSet[i] for i in reduced_index]
    print(reduced_data)
   #----------------------------------------------------------------------------------------------------------
    
    n_features = int(sqrt(len(reduced_data[0])-1))

    params = {
        'n_trees': 20                # number of trees to be built
        ,'sample_size': 1          # fraction to be randomly sampled for each tree
        ,'n_features':  n_features   # number of features to be randomly selected to evaluate for each split
        ,'max_depth': 9            # maximum depth of each tree
        ,'min_size': 1               # minimum records number after split
    }
    for n_trees in [5]:
        params['n_trees'] = n_trees
        scores = evaluate_algorithm(reduced_data, random_forest, params, n_folds=2)
        print('Trees: %d' % n_trees)
        print('Scores: %s' % scores)
        print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
        
        
   

numer of samples: 5749
[4384 5696 2941 ...  582 2219 1150]
[[0.888888888888889, 2, 0.0, 2, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0], [0.222222222222222, 2, 1.0, 2, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0], [1.0, 2, 0.0909090909090909, 2, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0], [0.571428571428571, 2, 1.0, 2, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0], [0.285714285714286, 2, 1.0, 2, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0], [1.0, 2, 1.0, 2, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], [1.0, 2, 0.0, 2, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0], [1.0, 2, 0.166666666666667, 2, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0], [0.75, 2, 0.0, 2, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0], [0.0, 2, 0.2, 2, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0], [1.0, 2, 0.2, 2, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0], [1.0, 2, 0.142857142857143, 2, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0], [1.0, 2, 1.0, 2, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], [1.0, 2, 1.0, 2, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], [0.888888888888889, 2, 1.0, 2, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], [0.333333333333333, 2, 0.571428571428571, 2, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.4, 2, 0.0, 2, 1.0, 1.0, 0.0

[{'left': {'left': {'left': 1.0, 'index': 7, 'right': 1.0, 'value': 0.0}, 'index': 5, 'right': {'left': 1.0, 'index': 7, 'right': 1.0, 'value': 0.0}, 'value': 1.0}, 'index': 2, 'right': {'left': {'left': 1.0, 'index': 0, 'right': 1.0, 'value': 0.0}, 'index': 0, 'right': {'left': {'left': {'left': {'left': 1.0, 'index': 4, 'right': 1.0, 'value': 1.0}, 'index': 1, 'right': {'left': 1.0, 'index': 1, 'right': 1.0, 'value': 2}, 'value': 2}, 'index': 2, 'right': {'left': {'left': 1.0, 'index': 5, 'right': 1.0, 'value': 0.0}, 'index': 5, 'right': {'left': {'left': 0.0, 'index': 0, 'right': 0.0, 'value': 1.0}, 'index': 4, 'right': {'left': 0.0, 'index': 4, 'right': 0.0, 'value': 1.0}, 'value': 1.0}, 'value': 1.0}, 'value': 1.0}, 'index': 8, 'right': {'left': {'left': 0.0, 'index': 2, 'right': {'left': 0.0, 'index': 5, 'right': 0.0, 'value': 1.0}, 'value': 1.0}, 'index': 0, 'right': {'left': {'left': 0.0, 'index': 1, 'right': 0.0, 'value': 2}, 'index': 7, 'right': {'left': {'left': 0.0, 'index'